In [2]:
import pandas as pd
import numpy as np

import fr_core_news_md
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
data = pd.read_csv("scraping/full.csv", sep=";", encoding="ANSI")
data.head()

,Date,Journal,Article
0,23/02/2022,Le Monde,Présidentielle 2022 : sur fond de crise ukrain...
1,23/02/2022,Le Monde,"Retour au septennat, législatives à mi-mandat,..."
2,23/02/2022,Le Monde,Jean-Pierre Chevènement va apporter son soutie...
3,23/02/2022,Le Monde,Marine Le Pen peine encore à trouver ses parra...
4,23/02/2022,Le Monde,Sibyle Veil : « J’ai choisi de nommer Adèle Va...


In [3]:
# 15 journals
data['Journal'].unique()

array(['Le Monde', 'Le Figaro', 'C News', '20 minutes', 'Les Echos',
       'France Info', 'Nouvel Obs', "L'Express", 'BFM TV', 'Marianne',
       'La Croix', 'Le Point', 'Valeurs Actuelles', 'Libération',
       "L'Humanité", nan], dtype=object)

In [9]:
# formatting datetime & dropping duplicates (ads, some headlines at several locations on websites)
data['Date'] = pd.to_datetime(data['Date'],infer_datetime_format=True)
data.dropna(how='any',inplace=True)
data.drop_duplicates(inplace=True,keep='first',subset=['Journal', 'Article'])

data.describe(include='all',datetime_is_numeric=True)

,Date,Journal,Article
count,56281,56281,56281
unique,NaN,15,55946
top,NaN,France Info,Elon Musk rachète Twitter pour 44 milliards de...
freq,NaN,11483,5
mean,2022-03-24 07:29:28.049608448,NaN,NaN
min,2022-02-23 00:00:00,NaN,NaN
25%,2022-03-09 00:00:00,NaN,NaN
50%,2022-03-24 00:00:00,NaN,NaN
75%,2022-04-09 00:00:00,NaN,NaN
max,2022-04-25 00:00:00,NaN,NaN


In [12]:
# removing punctuation but keeping special french characters, lowering
data['clean_articles'] = data['Article'].str.replace(r"[\W_]+", " ",regex=True)
data['clean_articles'] = data['clean_articles'].fillna('').apply(lambda x: x.lower())
display(data.head())
text_max_length = data.Article.map(lambda x: len(x)).max()
print(f'Max length of sequence : {text_max_length}')

,Date,Journal,Article,clean_articles
0,2022-02-23,Le Monde,Présidentielle 2022 : sur fond de crise ukrain...,présidentielle 2022 sur fond de crise ukrainie...
1,2022-02-23,Le Monde,"Retour au septennat, législatives à mi-mandat,...",retour au septennat législatives à mi mandat p...
2,2022-02-23,Le Monde,Jean-Pierre Chevènement va apporter son soutie...,jean pierre chevènement va apporter son soutie...
3,2022-02-23,Le Monde,Marine Le Pen peine encore à trouver ses parra...,marine le pen peine encore à trouver ses parra...
4,2022-02-23,Le Monde,Sibyle Veil : « J’ai choisi de nommer Adèle Va...,sibyle veil j ai choisi de nommer adèle van re...


Max length of sequence : 203


In [18]:
STOP_WORDS |= {"l", "j", "d"}
# appending stopwords not taken into account because of our text preprocessing (removing apostrophes)

In [20]:
# lemmatizing and removing stopwords
nlp = fr_core_news_md.load()
data['token_lemma'] = data['clean_articles'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) \
                    if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

data.head()

,Date,Journal,Article,clean_articles,token_lemma
0,2022-02-23,Le Monde,Présidentielle 2022 : sur fond de crise ukrain...,présidentielle 2022 sur fond de crise ukrainie...,présidentiel 2022 fond crise ukrainien hidalgo...
1,2022-02-23,Le Monde,"Retour au septennat, législatives à mi-mandat,...",retour au septennat législatives à mi mandat p...,septennat législatif mi mandat prérogative ely...
2,2022-02-23,Le Monde,Jean-Pierre Chevènement va apporter son soutie...,jean pierre chevènement va apporter son soutie...,jean pierre chevènement apporter soutien emman...
3,2022-02-23,Le Monde,Marine Le Pen peine encore à trouver ses parra...,marine le pen peine encore à trouver ses parra...,marine pen peine trouver parrainage
4,2022-02-23,Le Monde,Sibyle Veil : « J’ai choisi de nommer Adèle Va...,sibyle veil j ai choisi de nommer adèle van re...,sibyl veil choisir nommer adèle van reeth remp...


In [21]:
# vectorizing for LDA
vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (1, 2))
vectors = []

for index, row in data.iterrows():
    vectors.append(row['token_lemma'])

vectorised = vectorizer.fit_transform(vectors)
print(vectorised[0])

  (0, 164216)	1
  (0, 2077)	1
  (0, 83244)	1
  (0, 52608)	1
  (0, 211820)	1
  (0, 96725)	1
  (0, 107196)	1
  (0, 218596)	1
  (0, 55074)	1
  (0, 135272)	1
  (0, 164232)	1
  (0, 2504)	1
  (0, 83255)	1
  (0, 52719)	1
  (0, 212078)	1
  (0, 96780)	1
  (0, 107314)	1
  (0, 218620)	1
  (0, 55101)	1


In [22]:
# initializing LDA Model with 14 topics
lda_model = LatentDirichletAllocation(n_components = 14, random_state = 1, evaluate_every = -1,n_jobs = -1)
lda_output = lda_model.fit_transform(vectorised)

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]
df_topics = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each article
dominant_topic = (np.argmax(df_topics.values, axis=1)+1)
df_topics['Dominant_topic'] = dominant_topic

# join to original dataframe
data = pd.merge(data, df_topics['Dominant_topic'], left_index = True, right_index = True, how = 'outer')
display(data.head(3))

,Date,Journal,Article,clean_articles,token_lemma,Dominant_topic
0,2022-02-23,Le Monde,Présidentielle 2022 : sur fond de crise ukrain...,présidentielle 2022 sur fond de crise ukrainie...,présidentiel 2022 fond crise ukrainien hidalgo...,14.0
1,2022-02-23,Le Monde,"Retour au septennat, législatives à mi-mandat,...",retour au septennat législatives à mi mandat p...,septennat législatif mi mandat prérogative ely...,9.0
2,2022-02-23,Le Monde,Jean-Pierre Chevènement va apporter son soutie...,jean pierre chevènement va apporter son soutie...,jean pierre chevènement apporter soutien emman...,2.0


In [23]:
topic_count = data['Dominant_topic'].value_counts()
topic_count/sum(topic_count)*100
# topic distribution

9.0     14.916224
2.0     12.432260
11.0     7.883655
7.0      7.290205
12.0     6.853112
6.0      6.096196
3.0      6.035785
8.0      6.009133
5.0      5.930954
4.0      5.906078
1.0      5.861658
10.0     5.126064
13.0     4.914625
14.0     4.744052
Name: Dominant_topic, dtype: float64

In [ ]:
## TO DO LIST

# Explore LDA dominant topics
# Extract most common words for guided topic modeling with BERTopic... or do that from hierarchical topic modeling ?

# Transformer pipelines, CamemBERT ?

In [38]:
from bertopic import BERTopic

In [ ]:
# seed topic list to define

seed_topic_list = [["ex1", "ex2", "ex3", "ex4"],
                   ["su1", "su2", "su3", "su4"],
                   ["to1", "to2", "to3", "to4"]]

topic_model = BERTopic(seed_topic_list=seed_topic_list)
topics, probs = topic_model.fit_transform(data['token_lemma'])